In [5]:
!rm -r /kaggle/working/VisDrone/VisDrone2019-VID-test-dev

rm: cannot remove '/kaggle/working/VisDrone/VisDrone2019-VID-test-dev': No such file or directory


In [6]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

# Function to convert VisDrone annotations to YOLO format
def visdrone2yolo_batch(dir, batch_size=10):
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        x_center = (box[0] + box[2] / 2) * dw
        y_center = (box[1] + box[3] / 2) * dh
        width = box[2] * dw
        height = box[3] * dh
        return x_center, y_center, width, height

    labels_dir = dir / 'labels'
    labels_dir.mkdir(parents=True, exist_ok=True)

    sequences_dir = dir / 'sequences'
    annotations_dir = dir / 'annotations'

    # Ensure annotations directory exists
    if not annotations_dir.exists():
        print(f"Annotations directory not found: {annotations_dir}")
        return

    # Loop through all subdirectories (each representing a video) in the sequences directory
    video_dirs = list(sequences_dir.iterdir())
    total_videos = len(video_dirs)

    for batch_start in range(0, total_videos, batch_size):
        batch_end = min(batch_start + batch_size, total_videos)
        current_batch = video_dirs[batch_start:batch_end]

        for video_dir in current_batch:
            if video_dir.is_dir():
                video_name = video_dir.name
                annotation_file = annotations_dir / f"{video_name}.txt"

                if not annotation_file.exists():
                    print(f"Annotation file not found: {annotation_file}")
                    continue

                # Read annotation file once
                with open(annotation_file, 'r') as file:
                    annotations = [x.split(',') for x in file.read().strip().splitlines()]

                pbar = tqdm(list(video_dir.glob('*.jpg')), desc=f'Converting {video_dir}')

                for image_file in pbar:
                    frame_number = int(image_file.stem.split('_')[-1])  # Extract frame number
                    img_size = Image.open(image_file).size
                    lines = []

                    # Filter annotations for the current frame number
                    frame_annotations = [row for row in annotations if int(row[0]) == frame_number]

                    for row in frame_annotations:
                        if row[4] == '0':  # VisDrone 'ignored regions' class 0
                            continue
                        cls = int(row[5]) - 1  # Adjust class index
                        if cls >= 0 and cls <= 9:  # Ensure class is within the valid range
                            box = tuple(map(int, row[1:5]))  # Adjusted to use correct box indices
                            yolo_box = convert_box(img_size, box)
                            lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in yolo_box)}\n")

                    # Save YOLO formatted labels
                    label_path = labels_dir / f"{video_name}_{frame_number:06d}.txt"
                    with open(label_path, 'w') as fl:
                        fl.writelines(lines)

                # Remove processed annotation file to free up space
                os.remove(annotation_file)
                print(f"Deleted annotation file: {annotation_file}")

# Convert annotations for the datasets
root_dir = Path('/kaggle/working/VisDrone')
datasets = ['VisDrone2019-VID-train/VisDrone2019-VID-train', 'VisDrone2019-VID-val/VisDrone2019-VID-val']

for d in datasets:
    visdrone2yolo_batch(root_dir / d)  # Convert VisDrone annotations to YOLO labels

print("Conversion complete.")

Converting /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/sequences/uav0000222_03150_v: 100%|██████████| 421/421 [00:01<00:00, 265.61it/s]


Deleted annotation file: /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/annotations/uav0000222_03150_v.txt


Converting /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/sequences/uav0000140_01590_v: 100%|██████████| 256/256 [00:00<00:00, 320.31it/s]


Deleted annotation file: /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/annotations/uav0000140_01590_v.txt


Converting /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/sequences/uav0000352_05980_v: 100%|██████████| 196/196 [00:00<00:00, 248.67it/s]


...

Deleted annotation file: /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/annotations/uav0000339_00001_v.txt


Converting /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/sequences/uav0000086_00000_v: 100%|██████████| 464/464 [00:02<00:00, 217.61it/s]


Deleted annotation file: /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/annotations/uav0000086_00000_v.txt


Converting /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/sequences/uav0000117_02622_v: 100%|██████████| 349/349 [00:01<00:00, 282.10it/s]

Deleted annotation file: /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/annotations/uav0000117_02622_v.txt
Conversion complete.


In [7]:
!rm -r /kaggle/working/VisDrone/VisDrone2019-VID-val/VisDrone2019-VID-val/annotations

In [8]:
!rm -r /kaggle/working/VisDrone/VisDrone2019-VID-train/VisDrone2019-VID-train/annotations

In [9]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import yaml

# Define YAML content
content = {
    'path': '/kaggle/working/VisDrone',  # dataset root dir
    'train': '/kaggle/working/VisDrone/VisDrone-train/Sequences/',  # train images
    'val': '/kaggle/working/VisDrone/VisDrone-val/Sequences/',  # val images
    'test': '/kaggle/working/VisDrone/VisDrone-test/Sequences/',  # test images
    'names': {
        0: 'pedestrian',
        1: 'people',
        2: 'bicycle',
        3: 'car',
        4: 'van',
        5: 'truck',
        6: 'tricycle',
        7: 'awning-tricycle',
        8: 'bus',
        9: 'motor'
    }
}

# Define the path to save the YAML file
yaml_file_path = '/kaggle/working/dataset.yaml'

# Create the directory if it does not exist
yaml_dir = os.path.dirname(yaml_file_path)
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)

# Write the YAML content to the file
with open(yaml_file_path, 'w') as file:
    yaml.dump(content, file, default_flow_style=False)

print(f'YAML file created at: {yaml_file_path}')

# Function to convert VisDrone annotations to YOLO format
def visdrone2yolo(dir_images, dir_annotations):
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    label_dir = dir_annotations.parent / 'labels'
    label_dir.mkdir(parents=True, exist_ok=True)  # Make labels directory

    pbar = tqdm(dir_annotations.glob('*.txt'), desc=f'Converting {dir_annotations}')
    for f in pbar:
        img_filename = f.stem.split('_')[0] + '.jpg'
        img_path = None
        for img_dir in dir_images.glob('*/'):
            possible_img_path = img_dir / img_filename
            if possible_img_path.exists():
                img_path = possible_img_path
                break
        
        if img_path is None:
            continue
        
        img_size = Image.open(img_path).size
        lines = []
        with open(f, 'r') as file:  # Read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
        # Save YOLO formatted labels
        label_path = label_dir / (f.stem + '.txt')
        with open(label_path, 'w') as fl:
            fl.writelines(lines)

# Convert annotations for the datasets
root_dir = Path('/kaggle/working/VisDrone')
datasets = [
    ('VisDrone-train/Sequences', 'VisDrone-train/annotations'),
    ('VisDrone-val/Sequences', 'VisDrone-val/annotations'),
    ('VisDrone-test/Sequences', 'VisDrone-test/annotations')
]

for images_dir, annotations_dir in datasets:
    visdrone2yolo(root_dir / images_dir, root_dir / annotations_dir)  # Convert VisDrone annotations to YOLO labels

print("Conversion complete.")

YAML file created at: /kaggle/working/dataset.yaml


Converting /kaggle/working/VisDrone/VisDrone-train/annotations: 0it [00:00, ?it/s]
Converting /kaggle/working/VisDrone/VisDrone-val/annotations: 0it [00:00, ?it/s]
Converting /kaggle/working/VisDrone/VisDrone-test/annotations: 0it [00:00, ?it/s]

Conversion complete.


In [10]:
import yaml

# Define YAML content
content = {
    'path': '/kaggle/working/VisDrone/Dataset',  # dataset root dir
    'train': '/kaggle/working/VisDrone/Dataset/images/train',  # train images
    'val': '/kaggle/working/VisDrone/Dataset/images/val',  # val images
    'names': {
        0: 'pedestrian',
        1: 'people',
        2: 'bicycle',
        3: 'car',
        4: 'van',
        5: 'truck',
        6: 'tricycle',
        7: 'awning-tricycle',
        8: 'bus',
        9: 'motorcycle'
    }
}

# Define the path to save the YAML file
yaml_file_path = '/kaggle/working/VisDrone.yaml'

# Create the directory if it does not exist
yaml_dir = os.path.dirname(yaml_file_path)
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)

# Write the YAML content to the file
with open(yaml_file_path, 'w') as file:
    yaml.dump(content, file, default_flow_style=False)

print(f'YAML file created at: {yaml_file_path}')

YAML file created at: /kaggle/working/VisDrone.yaml


In [11]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 605.5 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.7/863.7 kB 5.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.7
    Uninstalling widgetsnbextension-3.6.7:
      Successfully uninstalled widgetsnbextension-3.6.7
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip uninstall -y wandb

Found existing installation: wandb 0.17.4
Uninstalling wandb-0.17.4:
  Successfully uninstalled wandb-0.17.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ultralytics
from ultralytics import YOLO

model=YOLO('yolov8x.pt')
results=model.train(data='/kaggle/working/VisDrone.yaml',epochs=20 ,imgsz=640, amp=False, batch=8)

ModuleNotFoundError: No module named 'ultralytics'

In [1]:
import cv2
import os

def create_video_from_frames(input_folder, output_video_path, frame_rate=30):
    # Get all files in the folder
    files = sorted([f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))])
    
    # Read the first frame to get the dimensions
    first_frame = cv2.imread(os.path.join(input_folder, files[0]))
    height, width, layers = first_frame.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' for .mp4, 'XVID' for .avi
    video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for file in files:
        frame = cv2.imread(os.path.join(input_folder, file))
        video.write(frame)
    
    video.release()

def process_all_folders(parent_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each subfolder
    for subfolder in os.listdir(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)
        if os.path.isdir(subfolder_path):
            output_video_path = os.path.join(output_folder, f"{subfolder}.mp4")
            create_video_from_frames(subfolder_path, output_video_path)

# Set the parent folder where all location folders are stored
parent_folder = '/kaggle/input/visdrone2019-mot-test/VisDrone2019-MOT-test-challenge/sequences'
output_folder = '/kaggle/working/'

process_all_folders(parent_folder, output_folder)

In [3]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 1.3 MB/s eta 0:00:00ta 0:00:01


In [5]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 668.2 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 6.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import supervision as sv
from ultralytics import YOLO

model = YOLO("/kaggle/input/yolov8n-940/yolov8n_940.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

sv.process_video(
    source_path="/kaggle/working/uav0000320_03910_v.mp4",
    target_path="/kaggle/working/1.mp4",
    callback=callback
)

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.



0: 544x960 39 pedestrians, 1 people, 1 bicycle, 7 cars, 3 tricycles, 12 motors, 123.5ms
Speed: 20.9ms preprocess, 123.5ms inference, 320.2ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 39 pedestrians, 2 bicycles, 7 cars, 1 van, 1 truck, 1 tricycle, 1 awning-tricycle, 17 motors, 8.5ms
Speed: 7.3ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 51 pedestrians, 1 bicycle, 8 cars, 2 vans, 1 tricycle, 11 motors, 8.4ms
Speed: 5.7ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 39 pedestrians, 1 bicycle, 9 cars, 2 vans, 2 tricycles, 14 motors, 8.4ms
Speed: 4.6ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 47 pedestrians, 1 bicycle, 9 cars, 3 vans, 1 tricycle, 1 awning-tricycle, 12 motors, 8.5ms
Speed: 5.5ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 42 pedestrians, 1 bicycle, 12 